In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from py2neo import Graph, Node, Relationship
from googlesearch import search

In [4]:
# Load csv into df

medias_df = pd.read_csv('C:/Users/Jo/Documents/Tech/MediasLists/MondeDiplo/medias_francais.tsv', sep='	')
relations_df = pd.read_csv('C:/Users/Jo/Documents/Tech/MediasLists/MondeDiplo/relations_medias_francais.tsv', sep='	')


In [5]:
# CAREFUL, HAD TO REPLACE THE CORRECT '’'‘'' SIGNS IN LIGN 76
relations_df[76:78]

,id,origine,valeur,cible,source,datePublication,dateConsultation
76,77,Vincent Bolloré,"49,77",Financière de l‘Odet,NaN,NaN,NaN
77,78,Financière de l‘Odet,"64,4",Bolloré,NaN,NaN,NaN


In [6]:
g = Graph("bolt://localhost:7687", auth=("neo4j", "Password"))


In [7]:
# Push medias_df into db
for index, row in medias_df.iterrows():
    a = Node(row['typeLibelle'], name=row['nom'], rangChallenges=row["rangChallenges"], mediaType=row["mediaType"], mediaPeriodicite=row["mediaPeriodicite"], mediaEchelle=row["mediaEchelle"],commentaire=row["commentaire"] )
    a.__primarylabel__ = row['typeLibelle']
    a.__primarykey__ = "name"
    g.merge(a)

    

In [8]:
# Push relations_df into db
for index, row in relations_df.iterrows():
    node_a = g.nodes.match( name=row["origine"]).first()
    node_b = g.nodes.match( name=row["cible"]).first()
    rel = Relationship(node_a, "OWNS", node_b, valeur=row["valeur"], source=row["source"], datePublication=row["dateConsultation"])
    g.merge(rel)

In [10]:
# google name and returns first url in google search
ggs = []
for index, row in medias_df.iterrows():
    for i in search(row.nom, tld="fr",lang='fr', num=10, stop=1, pause=2):
        ggs.append(i)
medias_df['googlesearch'] = ggs

In [35]:
# to clean googlesearch column from bad links
medias_df['googlesearch'][medias_df['nom']=='Le Nouveau Monde']=""
medias_df['googlesearch'][medias_df['nom']=='Le Monde libre']=""
medias_df['googlesearch'][medias_df['nom']=='Prier']="https://www.magazine-prier.fr/"
medias_df['googlesearch'][medias_df['nom']=='AOL']="https://fr.wikipedia.org/wiki/America_Online"
medias_df['googlesearch'][medias_df['nom']=='Lagardère Active']="http://www.lagardere.com/activites/lagardere-active-2610.html"
medias_df['googlesearch'][medias_df['nom']=='Management']="https://www.prismashop.fr/tous-les-magazines/actualite-economique/management.html"
medias_df['googlesearch'][medias_df['nom']=='Ghosn Capital']=""
medias_df['googlesearch'][medias_df['nom']=='Les Augustins de l’Assomption']="https://www.assomption.org/fr"
medias_df['googlesearch'][medias_df['nom']=='Pèlerin']="https://www.lepelerin.com/"
medias_df['googlesearch'][medias_df['nom']=='Famille Hutin']="https://fr.wikipedia.org/wiki/Fran%C3%A7ois_R%C3%A9gis_Hutin"
medias_df['googlesearch'][medias_df['nom']=='Le Progrès']="https://www.leprogres.fr/"
medias_df['googlesearch'][medias_df['nom']=='L’Echo Charitois']=""
medias_df['googlesearch'][medias_df['nom']=='La Liberté']="https://actu.fr/liberte-caen/"
medias_df['googlesearch'][medias_df['nom']=='Le Petit Bleu']="https://www.petitbleu.fr/"
medias_df['googlesearch'][medias_df['nom']=='Famille Lemoine']=""
medias_df['googlesearch'][medias_df['nom']=='L’Eclair']="http://www.larepubliquedespyrenees.fr/"
medias_df['googlesearch'][medias_df['nom']=='Sept Jours à Brest']="https://www.letelegramme.fr/finistere/brest/"
medias_df['googlesearch'][medias_df['nom']=='Publifin']="https://www.publifin.be/"
medias_df['googlesearch'][medias_df['nom']=='Normandie Dimanche']="https://www.paris-normandie.fr/"
medias_df['googlesearch'][medias_df['nom']=='Benjamin et Ariane de Rothschild']="https://fr.wikipedia.org/wiki/Benjamin_de_Rothschild"
medias_df['googlesearch'][medias_df['nom']=='Lampsane Investissement SA']=""
medias_df['googlesearch'][medias_df['nom']=='Franck Julien']="https://en.wikipedia.org/wiki/Franck_Julien"
medias_df['googlesearch'][medias_df['nom']=='Edi Invest']=""
medias_df['googlesearch'][medias_df['nom']=='Jean Christophe Tortora']=""


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [39]:
medias_df.to_csv("medias_df_withGoogle", encoding='utf-8', index=False)

In [40]:
medias_df

,id,nom,typeLibelle,typeCode,rangChallenges,mediaType,mediaPeriodicite,mediaEchelle,commentaire,googlesearch
0,1,Claude Perdriel,Personne physique,1,359.0,NaN,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/Claude_Perdriel
1,2,Sophia Publications,Personne morale,2,NaN,NaN,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/Sophia_Publications
2,3,Groupe Perdriel,Personne morale,2,NaN,NaN,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/Groupe_Perdriel
3,4,L’histoire,Média,3,NaN,GPE,Mensuel,NaN,NaN,https://www.lhistoire.fr/
4,5,Historia,Média,3,NaN,GPE,Mensuel,NaN,NaN,https://www.historia.fr/
5,6,Challenges,Média,3,NaN,GPE,Hebdomadaire,NaN,NaN,https://www.challenges.fr/
6,7,Sciences & Avenir,Média,3,NaN,GPE,Mensuel,NaN,NaN,https://www.sciencesetavenir.fr/
7,8,L’Opinion,Média,3,NaN,GPE,Quotidien,NaN,NaN,https://www.lopinion.fr/
8,9,Prisa,Personne morale,2,NaN,NaN,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/Prisa
9,10,Xavier Niel,Personne physique,1,13.0,NaN,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/Xavier_Niel


In [9]:
from googlesearch import search

In [35]:
query = "L’histoire"

In [40]:
for i in search(query, tld="fr",lang='fr', num=10, stop=1, pause=2): print(i)

https://www.lhistoire.fr/


In [30]:
for j in search(query, tld="fr",lang='fr', num=10, stop=10, pause=2): 
    print(j) 

https://www.lhistoire.fr/
https://www.lhistoire.fr/archives
https://www.lhistoire.fr/abonnement
https://www.lhistoire.fr/user/login
https://www.lhistoire.fr/boutique
https://www.lhistoire.fr/contact
https://www.lhistoire.fr/rubrique/carte
https://fr.wikipedia.org/wiki/L%27Histoire
https://fr.wikipedia.org/wiki/Tout_sur_l%27histoire
http://www.histoire-france.net/
